In [1]:
import pandas as pd
import numpy as np
from fastai.tabular.all import *

In [2]:
df = pd.read_pickle("datasets/data_EDA.pkl")

In [3]:
# Variables to use to predict income and wealth1

cont_names = [
    "AGE",
    "IC1",
    "IC2",
    "IC3",
    "IC4",
    "IC5",
    "IC6",
    "IC7",
    "IC8",
    "IC9",
    "IC10",
    "IC11",
    "IC12",
    "IC13",
    "IC14",
    "RAMNTALL",
]

cat_names = [
    "TCODE",
    "STATE",
    "ZIP",
    "MAJOR",
    "MDMAUD",
    "DOMAIN",
    "HOMEOWNR",
    "NUMCHLD",
    "ageGroup",
    "GENDER",
    "WEALTH1",
    "WEALTH2",
    "MSA",
    "ADI",
    "DMA",
    
]

In [4]:
df.INCOME.unique()

array([nan, 6.0, 3.0, 1.0, 4.0, 2.0, 7.0, 5.0], dtype=object)

In [5]:
cont_names, cat_names = cont_cat_split(df, max_card=40, dep_var="INCOME")
df_train = df[~df.INCOME.isnull()].copy()
df_test = df[df.INCOME.isnull()].copy().drop(columns=["INCOME"])

In [6]:
df_train["INCOME"] = df_train.INCOME.astype(int)

In [7]:
df_train.shape

(73906, 481)

In [10]:
splits = RandomSplitter(valid_pct=0.3)(range_of(df_train))

to = TabularPandas(
    df_train,
    procs=[FillMissing, Categorify, Normalize],
    cat_names=cat_names,
    cont_names=cont_names,
    y_names="INCOME",
    y_block=CategoryBlock,
    splits=splits,
)

dls = to.dataloaders(bs=128)

In [11]:
learn = tabular_learner(dls, layers=[4096, 2048, 1024, 512, 256], metrics=accuracy)

In [12]:
learn.fit_one_cycle(3, 0.03)

epoch,train_loss,valid_loss,accuracy,time
0,1.730774,1.734492,0.272563,01:11
1,1.273779,1.759314,0.296604,01:14
2,0.748132,2.043633,0.293266,01:19


In [18]:
preds = learn.get_preds()

In [33]:
np.round(to_np(preds[1]),0)

array([[5],
       [0],
       [4],
       ...,
       [4],
       [4],
       [6]], dtype=int8)

In [46]:
to.classes

{'OSOURCE': (#858) ['#na#',' ','AAA','AAD','ABC','ABD','ABP','ABW','ACB','ACC'...],
 'TCODE': (#46) ['#na#',0,1,2,3,4,6,7,9,13...],
 'STATE': (#54) ['#na#','AA','AE','AK','AL','AP','AR','AZ','CA','CO'...],
 'ZIP': (#17124) ['#na#','01247','01746-','01754','01852','01876','02038','02043','02114','02116-'...],
 'MAILCODE': (#3) ['#na#',' ','B'],
 'PVASTATE': (#4) ['#na#',' ','E','P'],
 'DOB': (#825) ['#na#',Timestamp('1930-01-01 00:00:00'),Timestamp('1930-02-01 00:00:00'),Timestamp('1930-03-01 00:00:00'),Timestamp('1930-04-01 00:00:00'),Timestamp('1930-05-01 00:00:00'),Timestamp('1930-06-01 00:00:00'),Timestamp('1930-07-01 00:00:00'),Timestamp('1930-08-01 00:00:00'),Timestamp('1930-09-01 00:00:00')...],
 'NOEXCH': (#5) ['#na#',' ','0','1','X'],
 'RECINHSE': (#3) ['#na#',' ','X'],
 'RECP3': (#3) ['#na#',' ','X'],
 'RECPGVG': (#3) ['#na#',' ','X'],
 'RECSWEEP': (#3) ['#na#',' ','X'],
 'MDMAUD': (#28) ['#na#','C1CM','C1LM','C1MM','C2CM','C2LM','C2MM','C5CM','C5MM','C5TM'...],
 'DOMAIN': (#1

In [39]:
dl = learn.dls.test_dl(df_test)

KeyError: "['INCOME'] not in index"

In [16]:
preds = learn.get_preds(dl=dl, with_decoded=True, with_input=True)

AttributeError: 'NoneType' object has no attribute 'argmax'

In [44]:
to_np(preds[-1]).mean()

0.5151742929625106

In [36]:
preds[2].size()

AttributeError: 'NoneType' object has no attribute 'size'